# Creating command-line Tools

In [1]:
from ctapipe.core import Tool, Component
from ctapipe.core.traits import (Integer, Float, List, Dict,Unicode, TraitError, observe)
import logging
from time import sleep

see https://github.com/ipython/traitlets/blob/master/examples/myapp.py

## Setup:

Create a few `Component`s that we will use later in a `Tool`:

In [2]:
class MyComponent(Component):
    """ A Component that does stuff """
    value = Integer(default_value=-1, help="Value to use").tag(config=True)

    def do_thing(self):
        self.log.debug("Did thing")

# in order to have 2 of the same components at once
class SecondaryMyComponent(MyComponent):
    pass


class AdvancedComponent(Component):
    """ An advanced technique """

    value1 = Integer(default_value=-1, help="Value to use").tag(config=True)
    infile = Unicode(help="input file name").tag(config=True)
    outfile = Unicode(help="output file name").tag(config=True)

    @observe('outfile')
    def on_outfile_changed(self, change):
        self.log.warning("Outfile was changed to '{}'".format(change))

In [3]:
MyComponent()

value,-1,Value to use (default: -1)


In [4]:
AdvancedComponent()

infile,,input file name (default: )
outfile,,output file name (default: )
value1,-1,Value to use (default: -1)


## Now create an executable Tool that contains the Components

In [5]:
class MyTool(Tool):
    name="mytool"
    description="do some things and stuff"
    aliases = Dict(dict(infile='AdvancedComponent.infile',
                        iterations='MyTool.iterations'))

    # Which classes are registered for configuration
    classes = List([MyComponent, AdvancedComponent, SecondaryMyComponent])

    # local configuration parameters
    iterations = Integer(5,help="Number of times to run",allow_none=False).tag(config=True)

    def setup_comp(self):
        # when constructing Components, you must add them to the 
        # list of registered instances using add_component. This allows
        # the full configuration to be tracked
        self.comp = self.add_component(MyComponent(parent=self))
        self.comp2 = self.add_component(SecondaryMyComponent(parent=self))
        

    def setup_advanced(self):
        self.advanced = self.add_component(AdvancedComponent(parent=self))

    def setup(self):
        self.setup_comp()
        self.setup_advanced()

    def start(self):
        self.log.info("Performing {} iterations...".format(self.iterations))
        for ii in range(self.iterations):
            self.log.info("ITERATION {}".format(ii))
            self.comp.do_thing()
            self.comp2.do_thing()
            sleep(0.1)
            
    def finish(self):
        self.log.warning("Shutting down.")
    

## Get Help info

The following allows you to print the help info within a Jupyter notebook, but this same inforamtion would be displayed if the user types:
```
  mytool --help
```

In [6]:
tool=MyTool()

In [7]:
tool.print_help()

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--infile=<Unicode> (AdvancedComponent.infile)
    Default: ''
    input file name
--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--log-level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--config=<Unicode> (Tool.config_file)
    Default: ''
    name of a configuration file with parameters to load in addition to command-
    line parameters

To see all available configurables, use `--help-all`



The following  is equivalant to the user typing `mytool --help-all`

In [8]:
tool.print_help(classes=True)

do some things and stuff

Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--infile=<Unicode> (AdvancedComponent.infile)
    Default: ''
    input file name
--iterations=<Int> (MyTool.iterations)
    Default: 5
    Number of times to run
--log-level=<Enum> (Application.log_level)
    Default: 30
    Choices: (0, 10, 20, 30, 40, 50, 'DEBUG', 'INFO', 'WARN', 'ERROR', 'CRITICAL')
    Set the log level by value or name.
--config=<Unicode> (Tool.config_file)
    Default: ''
    name of a configuration file with parameters to load in addition to command-
    line parameters

Class parameters
----------------

Parameters are set from command-line arguments of the form:
`--Class.trait=value`. This line is evaluated in Python, so simple expressions
are allowed, e.g.:: `--C.a='range(3)'` For setting C.a=[0,1,2].

MyTool options
-------------

## Run the tool

here we pass in argv since it is a Notebook, but if argv is not specified it's read from `sys.argv`, so the following is the same as running:

```sh
mytool --log_level=INFO --infile bork.txt --iterations=3
```

In [9]:
tool.run(argv=[])

INFO [MyTool] (tool/initialize): ctapipe version 0.7.0.post132+git12d768d


INFO [MyTool] (tool/run): Starting: mytool


INFO [MyTool] (tool/run): CONFIG: {'MyTool': {'config_file': '', 'iterations': 5, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_format': '%(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s', 'log_level': 20}, 'MyComponent': {'value': -1}, 'SecondaryMyComponent': {'value': -1}, 'AdvancedComponent': {'infile': '', 'outfile': '', 'value1': -1}}


INFO [MyTool] (<ipython-input-5-6ca0c3418416>/start): Performing 5 iterations...


INFO [MyTool] (<ipython-input-5-6ca0c3418416>/start): ITERATION 0


...

INFO [MyTool] (<ipython-input-5-6ca0c3418416>/start): ITERATION 3


INFO [MyTool] (<ipython-input-5-6ca0c3418416>/start): ITERATION 4


WARNING [MyTool] (<ipython-input-5-6ca0c3418416>/finish): Shutting down.


INFO [MyTool] (tool/run): Finished: mytool


INFO [MyTool] (tool/run): Output: 


In [10]:
tool.log_format = "%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s" 
tool.run(argv=['--log-level','INFO','--infile','bork.txt','--iterations','3'])

2020-01-29 16:27:59 : INFO [MyTool initialize] ctapipe version 0.7.0.post132+git12d768d


2020-01-29 16:27:59 : INFO [MyTool run] Starting: mytool


2020-01-29 16:27:59 : INFO [MyTool run] CONFIG: {'MyTool': {'config_file': '', 'iterations': 3, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s', 'log_level': 20}, 'MyComponent': {'value': -1}, 'SecondaryMyComponent': {'value': -1}, 'AdvancedComponent': {'infile': 'bork.txt', 'outfile': '', 'value1': -1}}


2020-01-29 16:27:59 : INFO [MyTool start] Performing 3 iterations...


2020-01-29 16:27:59 : INFO [MyTool start] ITERATION 0


...

2020-01-29 16:27:59 : INFO [MyTool start] ITERATION 2


2020-01-29 16:27:59 : WARNING [MyTool finish] Shutting down.


2020-01-29 16:27:59 : INFO [MyTool run] Finished: mytool


2020-01-29 16:27:59 : INFO [MyTool run] Output: 


2020-01-29 16:27:59 : INFO [MyTool run] Output: 


here we change the log-level to DEBUG:

In [11]:
tool.run(argv=['--log-level','DEBUG','--infile','bork.txt'])

2020-01-29 16:27:59 : INFO [MyTool initialize] ctapipe version 0.7.0.post132+git12d768d


2020-01-29 16:27:59 : INFO [MyTool run] Starting: mytool


2020-01-29 16:27:59 : INFO [MyTool run] CONFIG: {'MyTool': {'config_file': '', 'iterations': 3, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s', 'log_level': 10}, 'MyComponent': {'value': -1}, 'SecondaryMyComponent': {'value': -1}, 'AdvancedComponent': {'infile': 'bork.txt', 'outfile': '', 'value1': -1}}


2020-01-29 16:27:59 : INFO [MyTool start] Performing 3 iterations...


2020-01-29 16:27:59 : INFO [MyTool start] ITERATION 0


...

2020-01-29 16:27:59 : INFO [MyTool run] Finished: mytool


2020-01-29 16:27:59 : INFO [MyTool run] Output: 


2020-01-29 16:27:59 : INFO [MyTool run] Output: 


2020-01-29 16:27:59 : INFO [MyTool run] Output: 


2020-01-29 16:27:59 : DEBUG [MyTool run] PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "53fe4815-9dba-47ba-acfe-64e662567602",
      "start": {
         "time_utc": "2020-01-29T16:27:58.341"
      },
      "stop": {
         "time_utc": "2020-01-29T16:27:59.115"
      },
      "system": {
         "ctapipe_version": "0.7.0.post132+git12d768d",
         "ctapipe_resources_version": "0.2.18",
         "pyhessio_version": "2.1.1",
         "eventio_version": "1.0.0",
         "ctapipe_svc_path": null,
         "executable": "/home/travis/virtualenv/python3.6.7/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "ELF",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "travis-job-d399fa77-c587-4472-acce-4628cc3d4a24",
            "version": "#29~16.04.1-Ubuntu SMP Tue Feb 12 16:31:10 UTC 2019",
            "system": "Linux",
            "release": "4.15.0-1028-g

you can also set parameters directly in the class, rather than using the argument/configfile parser. This is useful if you are calling the Tool from a script rather than the command-line

In [12]:
tool.iterations = 1
tool.log_level = 0
tool.run('')

2020-01-29 16:27:59 : WARNING [MyTool finish] Shutting down.


see what happens when a value is set that is not of the correct type:

In [13]:
try:
    tool.iterations = "badval"
except TraitError as E:
    print("bad value:",E)

bad value: The 'iterations' trait of a MyTool instance must be an int, but a value of 'badval' <class 'str'> was specified.


Example of what happens when you change a parameter that is being "observed" in a class. It's handler is called:

In [14]:
tool.advanced.outfile = "Another.txt"

2020-01-29 16:27:59 : WARNING [MyTool.AdvancedComponent on_outfile_changed] Outfile was changed to '{'name': 'outfile', 'old': '', 'new': 'Another.txt', 'owner': <__main__.AdvancedComponent object at 0x7f24a68e4400>, 'type': 'change'}'


we see that the handler for `outfile` was called, and it receive a change dict that shows the old and new values.

create a tool using a config file:

In [15]:
!cat Tools.json

{
    "version": 1.0,
    "MyTool": {"log_level":"DEBUG"},
    "AdvancedComponent": {"infile": "something.txt"}
}


In [16]:
tool2 = MyTool()

In [17]:
tool2.run(argv=['--config','Tools.json'])

2020-01-29 16:28:00 : INFO [MyTool initialize] ctapipe version 0.7.0.post132+git12d768d


2020-01-29 16:28:00 : INFO [MyTool run] Starting: mytool


2020-01-29 16:28:00 : INFO [MyTool run] CONFIG: {'MyTool': {'config_file': 'Tools.json', 'iterations': 5, 'log_datefmt': '%Y-%m-%d %H:%M:%S', 'log_format': '%(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s', 'log_level': 10}, 'MyComponent': {'value': -1}, 'SecondaryMyComponent': {'value': -1}, 'AdvancedComponent': {'infile': 'something.txt', 'outfile': '', 'value1': -1}}


2020-01-29 16:28:00 : INFO [MyTool start] Performing 5 iterations...


2020-01-29 16:28:00 : INFO [MyTool start] ITERATION 0


...

2020-01-29 16:28:00 : INFO [MyTool run] Output: 


2020-01-29 16:28:00 : INFO [MyTool run] Output: 


2020-01-29 16:28:00 : INFO [MyTool run] Output: 


2020-01-29 16:28:00 : INFO [MyTool run] Output: 


2020-01-29 16:28:00 : DEBUG [MyTool run] PROVENANCE: '[
   {
      "activity_name": "mytool",
      "activity_uuid": "53fe4815-9dba-47ba-acfe-64e662567602",
      "start": {
         "time_utc": "2020-01-29T16:27:58.341"
      },
      "stop": {
         "time_utc": "2020-01-29T16:27:59.115"
      },
      "system": {
         "ctapipe_version": "0.7.0.post132+git12d768d",
         "ctapipe_resources_version": "0.2.18",
         "pyhessio_version": "2.1.1",
         "eventio_version": "1.0.0",
         "ctapipe_svc_path": null,
         "executable": "/home/travis/virtualenv/python3.6.7/bin/python",
         "platform": {
            "architecture_bits": "64bit",
            "architecture_linkage": "ELF",
            "machine": "x86_64",
            "processor": "x86_64",
            "node": "travis-job-d399fa77-c587-4472-acce-4628cc3d4a24",
            "version": "#29~16.04.1-Ubuntu SMP Tue Feb 12 16:31:10 UTC 2019",
            "system": "Linux",
            "release": "4.15.0-1028-g

In [18]:
print(tool2.advanced.infile)

something.txt


In [19]:
print(tool2.config)

{'MyTool': {'config_file': 'Tools.json', 'log_level': 'DEBUG'}, 'AdvancedComponent': {'infile': 'something.txt'}}


In [20]:
tool2.is_setup

True

In [21]:
tool3 = MyTool()

In [22]:
tool3.is_setup

False

In [23]:
tool3.initialize(argv=[])

2020-01-29 16:28:00 : INFO [MyTool initialize] ctapipe version 0.7.0.post132+git12d768d


In [24]:
tool3.is_setup

False

In [25]:
tool3

config_file,,name of a configuration file with parameters to load in addition to command-line parameters (default: )
iterations,5,Number of times to run (default: 5)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_format,%(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s,The Logging format template (default: %(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s)
log_level,20,Set the log level by value or name. (default: 30)


In [26]:
tool

config_file,,name of a configuration file with parameters to load in addition to command-line parameters (default: )
iterations,1,Number of times to run (default: 5)
log_datefmt,%Y-%m-%d %H:%M:%S,The date format used by logging formatters for %(asctime)s (default: %Y-%m-%d %H:%M:%S)
log_format,%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s,The Logging format template (default: %(levelname)s [%(name)s] (%(module)s/%(funcName)s): %(message)s)
log_level,0,Set the log level by value or name. (default: 30)


In [27]:
tool.comp2

value,-1,Value to use (default: -1)


## Getting the configuration of an instance

In [28]:
tool.get_current_config()

{'MyTool': {'config_file': '',
  'iterations': 1,
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s',
  'log_level': 0},
 'MyComponent': {'value': -1},
 'SecondaryMyComponent': {'value': -1},
 'AdvancedComponent': {'infile': 'bork.txt',
  'outfile': 'Another.txt',
  'value1': -1}}

In [29]:
tool.iterations = 12
tool.get_current_config()

{'MyTool': {'config_file': '',
  'iterations': 12,
  'log_datefmt': '%Y-%m-%d %H:%M:%S',
  'log_format': '%(asctime)s : %(levelname)s [%(name)s %(funcName)s] %(message)s',
  'log_level': 0},
 'MyComponent': {'value': -1},
 'SecondaryMyComponent': {'value': -1},
 'AdvancedComponent': {'infile': 'bork.txt',
  'outfile': 'Another.txt',
  'value1': -1}}

## Writing a Sample Config File

In [30]:
print(tool.generate_config_file())

# Configuration file for mytool.

#------------------------------------------------------------------------------
# Application(SingletonConfigurable) configuration
#------------------------------------------------------------------------------

## This is an application.

## The date format used by logging formatters for %(asctime)s
#c.Application.log_datefmt = '%Y-%m-%d %H:%M:%S'

## The Logging format template
#c.Application.log_format = '[%(name)s]%(highlevel)s %(message)s'

## Set the log level by value or name.
#c.Application.log_level = 30

#------------------------------------------------------------------------------
# Tool(Application) configuration
#------------------------------------------------------------------------------

## This is an application.

## name of a configuration file with parameters to load in addition to command-
#  line parameters
#c.Tool.config_file = ''

## The Logging format template
#c.Tool.log_format = '%(levelname)s [%(name)s] (%(module)s/%(funcNa